In [ ]:
# from: https://github.com/buomsoo-kim/Word-embedding-with-Python/blob/master/doc2vec/source%20code/doc2vec.ipynb

In [4]:
from rose_wrapper.rose import Rose
import pandas as pd
import datetime

from pprint import pprint
import os
import matplotlib.pyplot as plt
from nltk.tokenize import sent_tokenize

import re
from nltk import PorterStemmer, WordNetLemmatizer, word_tokenize
from nltk.corpus import stopwords
from os import listdir
from os.path import isfile, isdir, join
import numpy as np
import numpy
import sys
import getopt
import codecs
import time
import csv
import io
import pickle

In [5]:
import re
import numpy as np
import tensorflow as tf

from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument
from nltk.corpus import gutenberg
from multiprocessing import Pool
from scipy import spatial


In [79]:
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.mixture import GaussianMixture
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
from itertools import count
import seaborn as sn
import pandas as pd
from matplotlib.pyplot import figure

In [6]:
PATH = '/Users/krishnarao/Desktop/Black Snow/data'

In [8]:
direct =  PATH + '/financialdata/'


# functions
def tokenize(txt):
    """Simple white space tokenizer."""
    return txt.split()


def scores(row):
    """Change sentiment groups to numeric values."""
    if row["cla"] == "neutral":
        val = 0
    elif row["cla"] == "positive":
        val = 1
    else:
        val = -1

    return val


def accuracy(pred, actual):
    """Compute model accuracy."""
    return sum(pred == actual) / len(pred)

def subsample(df, n, seed):
    """Create subsample with oversampling."""
    con = df["sentiment"] == 0
    df2 = df.loc[con].sample(n, replace = True, random_state = seed)

    con = df["sentiment"] == 1
    df2 = df2.append(df.loc[con].sample(n, replace = True, random_state = seed))

    con = df["sentiment"] == -1
    df2 = df2.append(df.loc[con].sample(n, replace = True, random_state = seed))
    return df2


# lexical based
def wordcount(words, dct):
    """Count words in dictionary."""
    from collections import Counter

    counting = Counter(words)
    count = []

    for key, value in counting.items():
        if key in dct:
            count.append([key, value])

    return count


def negwordcount(words, dct, negdct, lngram):
    """Count negated words in dictionary."""
    from nltk.util import ngrams

    mid = int(lngram / 2)
    ng = ngrams(words, lngram)
    nglist = []

    for grams in ng:
        nglist.append(grams)

    keeper = []
    n = len(nglist)
    i = 1
    for grams in nglist:
        if n - i < int(lngram / 2):
            mid = mid + 1

        if grams[mid] in dct:
            for j in grams:
                if j in negdct:
                    keeper.append(grams[mid])
                    break

        i = i + 1

    count = wordcount(keeper, dct)

    return count


def findneg(word, wcneg):
    """Find negation word."""
    keywordneg = 0

    for j in range(0, len(wcneg)):
        if word in wcneg[j][0]:
            keywordneg = wcneg[j][1]
            break

    return keywordneg
def lexcnt(txt, pos_dct, neg_dct, negat_dct, lngram):
    """Count words and negated words in dictionary."""
    from nltk import word_tokenize
    txt = word_tokenize(txt)
    # Count words in lexicon
    pos_wc = wordcount(txt, pos_dct)
    pos_wc = [cnt[1] for cnt in pos_wc]
    pos_wc = sum(pos_wc)

    neg_wc = wordcount(txt, neg_dct)
    neg_wc = [cnt[1] for cnt in neg_wc]
    neg_wc = sum(neg_wc)

    # Count negated words in lexicon
    pos_wcneg = negwordcount(txt, pos_dct, negat_dct, lngram)
    pos_wcneg = [cnt[1] for cnt in pos_wcneg]
    pos_wcneg = sum(pos_wcneg)

    neg_wcneg = negwordcount(txt, neg_dct, negat_dct, lngram)
    neg_wcneg = [cnt[1] for cnt in neg_wcneg]
    neg_wcneg = sum(neg_wcneg)

    pos = pos_wc - (pos_wcneg) + neg_wcneg
    neg = neg_wc - (neg_wcneg) + pos_wcneg

    if pos > neg:
        out = 1
    elif pos < neg:
        out = -1
    else:
        out = 0

    return out

# read data

# change directory
os.chdir(direct + "financial_phrase_bank")

# load phrasebank
phrasebank = pickle.load(open("lem_Sentences_66Agree.p", "rb"))

sentences = pd.Series(phrasebank[0])
classif = pd.Series(phrasebank[1])
ident = pd.Series(range(0, len(classif)))

df = pd.DataFrame({"lemma": sentences,
                   "cla": classif,
                   })

df["sentiment"] = df.apply(scores, axis = 1)

negat_dct = ["n't", "not", "never", "no", "neither", "nor", "none"]
lngram = 7



# write out test and train data sets
# print df
train = open((PATH + "/train.txt"),"w+")
test = open((PATH + "/test.txt"),"w+")
for i in range(len(df)):
    if not (np.random.rand() >= 0.8):
        train.write(" "*5 + str(i) + "\t" + df.iloc[i]["lemma"] + "\t" + str(df.iloc[i]["sentiment"]) + "\n")
    else:
        test.write(" "*5 + str(i) + "\t" + df.iloc[i]["lemma"] + "\t" + str(df.iloc[i]["sentiment"]) + "\n")
train.close()
test.close()

In [13]:
WORD_COUNT_THRESHOLD = 5

chars = ['{','}','#','%','&','\(','\)','\[','\]','<','>',',', '!', '.', ';', 
'?', '*', '\\', '\/', '~', '_','|','=','+','^',':','\"','\'','@','-']

def stem(word):
   regexp = r'^(.*?)(ing|ly|ed|ious|ies|ive|es|s|ment)?$'
   stem, suffix = re.findall(regexp, word)[0]
   return stem

def unique(a):
   """ return the list with duplicate elements removed """
   return list(set(a))

def intersect(a, b):
   """ return the intersection of two lists """
   return list(set(a) & set(b))

def union(a, b):
   """ return the union of two lists """
   return list(set(a) | set(b))

def get_files(mypath):
   return [ f for f in listdir(mypath) if isfile(join(mypath,f)) ]

def get_dirs(mypath):
   return [ f for f in listdir(mypath) if isdir(join(mypath,f)) ]

def tokenize_corpus(path, train=True):

  porter = PorterStemmer() # also lancaster stemmer
  wnl = WordNetLemmatizer()
  stopWords = stopwords.words("english")
  classes = []
  samples = []
  docs = []
  if train == True:
    words = {}
  f = open(path, 'r')
  lines = f.readlines()

  for line in lines:
    classes.append(line.rsplit()[-1])
    samples.append(line.rsplit()[0])
    raw = line
    raw = ' '.join(raw.rsplit()[1:-1])
    # remove noisy characters; tokenize
    raw = re.sub('[%s]' % ''.join(chars), ' ', raw)
    tokens = word_tokenize(raw)
    tokens = [w.lower() for w in tokens]
    tokens = [w for w in tokens if w not in stopWords]
    tokens = [wnl.lemmatize(t) for t in tokens]
    tokens = [porter.stem(t) for t in tokens]   
    if train == True:
     for t in tokens: 
         try:
             words[t] = words[t]+1
         except:
             words[t] = 1
    docs.append(tokens)

  if train == True:
     return(docs, classes, samples, words)
  else:
     return(docs, classes, samples)

def wordcount_filter(words, num=WORD_COUNT_THRESHOLD):
   keepset = []
   for k in words.keys():
       if(words[k] > num):
           keepset.append(k)
   print ("Vocab length:", len(keepset))
   return(sorted(set(keepset)))

def find_wordcounts(docs, vocab):
   bagofwords = numpy.zeros(shape=(len(docs),len(vocab)), dtype=numpy.uint8)
   vocabIndex={}
   for i in range(len(vocab)):
      vocabIndex[vocab[i]]=i

   for i in range(len(docs)):
       doc = docs[i]

       for t in doc:
          index_t=vocabIndex.get(t)
          if index_t != None:
             bagofwords[i,index_t]=bagofwords[i,index_t]+1

   print ("Finished find_wordcounts for:", len(docs), "docs")
   return(bagofwords)

start_time = time.time()

outputf = 'v2'
vocabf = 'v1_vocab_' + str(WORD_COUNT_THRESHOLD) + '.txt'

path = PATH #'/Users/chris/findata/data'
traintxt = path+"/train.txt"
testtxt  = path+"/test.txt"
print ('Path:', path)
print ('Training data:', traintxt)
print ('Test data', testtxt)

# Tokenize training data (if training vocab doesn't already exist):
if (not vocabf):
    word_count_threshold = WORD_COUNT_THRESHOLD
    (docs, classes, samples, words) = tokenize_corpus(traintxt, train=True)
    (docsTest, classesTest, samplesTest) = tokenize_corpus(testtxt, train=False)
    vocab = wordcount_filter(words, num=word_count_threshold)
    # Write new vocab file
    vocabf = outputf+"_vocab_"+str(word_count_threshold)+".txt"
    outfile = codecs.open(path+"/"+vocabf, 'w',"utf-8-sig")
    outfile.write("\n".join(vocab))
    outfile.close()
else:
    word_count_threshold = WORD_COUNT_THRESHOLD
    (docs, classes, samples) = tokenize_corpus(traintxt, train=False)
    (docsTest, classesTest, samplesTest) = tokenize_corpus(testtxt, train=False)
    vocabfile = open(path+"/"+vocabf, 'r')
    vocab = [line.rstrip('\n') for line in vocabfile]
    vocabfile.close()

print ('Vocabulary file:', path+"/"+vocabf)

# Get bag of words:
bow = find_wordcounts(docs, vocab)
bowTest = find_wordcounts(docsTest, vocab)
# Check: sum over docs to check if any zero word counts
print ("Doc with smallest number of words in vocab has:", min(numpy.sum(bow, axis=1)))
print ("Test Doc with smallest number of words in vocab has:", min(numpy.sum(bowTest, axis=1)))

# Write bow file
'''with open(path+"/"+outputf+"_bag_of_words_"+str(word_count_threshold)+".csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(bow)'''
with open(path+"/"+outputf+"_test_bag_of_words_"+str(word_count_threshold)+".csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(bowTest)
# Write classes
'''outfile = open(path+"/"+outputf+"_classes_"+str(word_count_threshold)+".txt", 'w')
outfile.write("\n".join(classes))
outfile.close()'''
outfile = open(path+"/"+outputf+"_test_classes_"+str(word_count_threshold)+".txt", 'w')
outfile.write("\n".join(classesTest))
outfile.close()

# Write samples
'''outfile = open(path+"/"+outputf+"_samples_class_"+str(word_count_threshold)+".txt", 'w')
outfile.write("\n".join(samples))
outfile.close()'''
outfile = open(path+"/"+outputf+"_test_samples_class_"+str(word_count_threshold)+".txt", 'w')
outfile.write("\n".join(samplesTest))
outfile.close()

print ('Output files:', path+"/"+outputf+"*")

# Runtime
print ('Runtime:', str(time.time() - start_time))

import csv, numpy as np
from numpy import genfromtxt
import random

Path: /Users/krishnarao/Desktop/Black Snow/data
Training data: /Users/krishnarao/Desktop/Black Snow/data/train.txt
Test data /Users/krishnarao/Desktop/Black Snow/data/test.txt
Vocabulary file: /Users/krishnarao/Desktop/Black Snow/data/v1_vocab_5.txt
Finished find_wordcounts for: 3353 docs
Finished find_wordcounts for: 864 docs
Doc with smallest number of words in vocab has: 0
Test Doc with smallest number of words in vocab has: 0
Output files: /Users/krishnarao/Desktop/Black Snow/data/v2*
Runtime: 5.486937046051025


In [15]:
def openTxt(filename):
    file = open(filename,'r')
    newlist = file.read().split('\n')
    file.close()
    return newlist

# Open files with Words: 
vocabList = openTxt(PATH + '/v1_vocab_' + str(WORD_COUNT_THRESHOLD) + '.txt')
train = openTxt(PATH + '/train.txt')

In [16]:
train = openTxt(PATH + '/train.txt')

In [48]:
train_file = [] 
for elem in train: 
    tokenized_sent = re.sub(r'\W+', ' ', elem).split()[1:-1]
    train_file.append(tokenized_sent)

In [52]:
sentences = train_file.copy()

In [53]:
for i in range(len(sentences)):
    sentences[i] = TaggedDocument(words = sentences[i], tags = ['sent{}'.format(i)]) 

In [55]:

model = Doc2Vec(documents = sentences, dm = 1, vector_size = 100, window = 3, min_count = 1, epochs = 10, workers = Pool()._processes)


In [56]:
model.init_sims(replace = True)

In [59]:
model.save('reuters_sent_1_doc2vec_model')

In [58]:
model = Doc2Vec.load('reuters_sent_1_doc2vec_model')

In [60]:
v1 = model.infer_vector(sentences[1][0])    # in doc2vec, infer_vector() function is used to infer the vector embedding of a document
v2 = model.infer_vector(sentences[300][0])

In [62]:
def cosine_similarity(v1, v2):
    return 1 - spatial.distance.cosine(v1, v2)

In [63]:
cosine_similarity(v1, v2)

0.8291740417480469

In [136]:
word_embeddings = []
for sentence in sentences:
    word_embeddings.append(model.infer_vector(sentence[0]))

In [73]:
np.save('word_embeddings_1.npy', np.asarray(word_embeddings))

In [151]:
Y_tot_raw = [] 
for elem in train: 
    tokenized_sent = elem.split()[-1:]
    Y_tot_raw.append(tokenized_sent)

In [159]:
X_tot = word_embeddings
Y_tot = [int(elem) for elem in sum(Y_tot_raw , [])]

In [191]:
X_train = []
X_test = []
y_train = []
y_test = []

for i in range(len(Y_tot)):
    prop = np.random.rand()
    if prop >=0.8:
        X_test.append(X_tot[i])
        y_test.append(Y_tot[i])
    else:
        X_train.append(X_tot[i])
        y_train.append(Y_tot[i])

In [186]:
def calcCrossval(model):
    scores = cross_val_score(model, X_train, y_train, cv=10)
    print("Cross Val Training Accuracy: " + str(round(scores.mean(),3)) + ", STD: " + str(round(scores.std()*2,3)))
    return scores

# Create a class that contains a model and all its variables
# In this case, the 'class' is just a variable of a model
# that contains the model's relevant variables such as fpr, tpr, etc. 
class modDetails:    
    def __init__(self, predictionsTrain, predictionsTest, 
                            classifier, train_cvscores, name):
        self.predictionsTrain = predictionsTrain
        self.predictionsTest = predictionsTest
        self.classifier = classifier
        self.train_cvscores = train_cvscores
        self.name = name

print('done')

done


In [187]:
def accur(prediction, y):
    n1 = len(prediction)
    n2 = len(y)
    if not (n1 == n2):
        return -10000
    count = 0
    for i in range(n1):
        if y[i] == prediction[i]:
            count = count + 1
    return count/n1

def runModel2(model,name):
    print(name + '-'*50 + ' \n')
    
    # Train the Classifier
    # And obtain 10-fold cross validation results of the training
    if name.find('GMM') >= 0: # if this is GMM, only fit with X_train
        tt = model.fit(X_train)
    else:
        tt = model.fit(X_train,y_train)
    
    train_cvscores = calcCrossval(model)
    
    print(str(type(tt)))
    
    # Calculate classifier accuracy on trained data and test data
    predictionsTrain = tt.predict(X_train)
    predictionsTest = tt.predict(X_test)
    
    print("Test accuracy:", accur(predictionsTest, y_test))
    print("Train accuracy:", accur(predictionsTrain, y_train))
    
    model_info = modDetails(predictionsTrain, predictionsTest, 
                            tt, train_cvscores, name)
    return model_info

print('done')

done


In [199]:
logreg2 = LogisticRegression(C=1, penalty='l1', tol=0.01, solver='saga', multi_class='multinomial')
logregL1_info = runModel2(logreg2, 'LogReg-L1')

LogReg-L1-------------------------------------------------- 

Cross Val Training Accuracy: 0.624, STD: 0.049
<class 'sklearn.linear_model.logistic.LogisticRegression'>
Test accuracy: 0.62339514978602
Train accuracy: 0.6300904977375565


In [210]:
rf = RandomForestClassifier(n_estimators=100, max_depth=2,random_state=0)
rf_info = runModel2(rf, 'Random Forest')

Random Forest-------------------------------------------------- 

Cross Val Training Accuracy: 0.633, STD: 0.064
<class 'sklearn.ensemble.forest.RandomForestClassifier'>
Test accuracy: 0.6148359486447932
Train accuracy: 0.6429110105580694


In [196]:
gbc = GradientBoostingClassifier()
gbc_info = runModel2(gbc, 'Gradient Boost')

Gradient Boost-------------------------------------------------- 

Cross Val Training Accuracy: 0.615, STD: 0.077
<class 'sklearn.ensemble.gradient_boosting.GradientBoostingClassifier'>
Test accuracy: 0.6333808844507846
Train accuracy: 0.8009049773755657
